<a href="https://colab.research.google.com/github/saravrbic/CUS1156_Lab2/blob/main/Project-Exam/Problem_set_08_Exam_Project_CUS615.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is part the of **Dr. Christoforos Christoforou's** course materials. You may not, nor may you knowingly allow others to reproduce or distribute lecture notes, course materials or any of their derivatives without the instructor's express written consent.

# Exam Project
**Professor**: Dr. Christoforos Christoforou

The objective of this project is to reproduce the data analysis method presented in the paper:

* **"Mining Big Data to Extract Patterns and Predict Real-Life Outcomes"**
by Michal Kosinski, Yilun Wang, Himabindu Lakkaraju, and Jure Leskovec.


You must implement the analysis using python code. This notebook provides you with the starting code to load the data and guides you through the requirement and each sub-task of the analysis. To complete this project you are expected to:

1. Carefully read the paper.
2. Refer the class lecture where this paper was discussed in detail.
3. Research various python libraries and utility functions needed to implement your analysis.
4. Complete all the challenges outlined in this notebook.



## Student Information 
Complete your information in the form provided below. This is visible in colab. 


In [ ]:
#@title Strudent Information 
student_name = "Sara Vrbic" #@param {type:"string"}
course_code = "CUS 615" #@param {type:"string"}


## Starter code - Configuring the analysis environment
The code cells below provide you a starting code to setup the environment for you to complete this project's tasks.

In particular, the code downloads the three dataset associated with this project and stores them in the folder `/content/sample_data/`. The data files from the paper comprise the `likes.csv` file, the `users.csv` file and the `users-likes.csv`; description of each data file is availabile in the paper.




In [161]:
#
# Run this sell to download the data for your analysis. 
#
!wget -O /content/sample_data/sample_dataset.zip https://osf.io/9m87k/download
!unzip /content/sample_data/sample_dataset.zip -d /content/sample_data/


--2023-05-03 00:08:41--  https://osf.io/9m87k/download
Resolving osf.io (osf.io)... 35.190.84.173
Connecting to osf.io (osf.io)|35.190.84.173|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://files.osf.io/v1/resources/fuqjg/providers/osfstorage/5e946850f1353503a3d52f24?action=download&direct&version=1 [following]
--2023-05-03 00:08:41--  https://files.osf.io/v1/resources/fuqjg/providers/osfstorage/5e946850f1353503a3d52f24?action=download&direct&version=1
Resolving files.osf.io (files.osf.io)... 35.186.214.196
Connecting to files.osf.io (files.osf.io)|35.186.214.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 314322145 (300M) [application/octet-stream]
Saving to: ‘/content/sample_data/sample_dataset.zip’

/content/sample_dat 100%[===================>] 299.76M  44.5MB/s    in 8.6s    

2023-05-03 00:08:50 (34.7 MB/s) - ‘/content/sample_data/sample_dataset.zip’ saved [314322145/314322145]

Archive:  /content/sample_data/sa

In [162]:
pip install factor_analyzer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [163]:
#
# You will likely need the following python packages when implementing this project.
# Add any additional package as you deem necessary 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds, eigs
from factor_analyzer import Rotator

# Additional package you want to use:


## Challenge 1 : Load the data into dataframes
In this challenge you are expected to load the data from the raw files into the following variables:
-  `users` : should reference a dataframe with the data in the `users.csv` file
-  `likes` : should reference a dataframe with the data in the `likes.csv` file
-  `ul` : should reference a dataframe with the data in `users.csv` file

Then print the top 10 rows of reach dataframe.

In [164]:

datafile1 = './sample_data/likes.csv'  
datafile2 =  './sample_data/users.csv'  
datafile3 = './sample_data/users-likes.csv'

likes = pd.read_csv(datafile1)
users = pd.read_csv(datafile2)
ul = pd.read_csv(datafile3)


In [165]:
likes.head(10)
users.head(10)
ul.head(10)

,userid,likeid
0,71bc7c0901488aec6d30f0add257e7c5,3c1636c878e6eb2acfd00c6b61086e38
1,978ab8e90c4d6ad1a48ef5c973b62f4d,feca46ddb8ef04f86172ace0cb7e004c
2,85123b0e358907725cf19a2cb0ec3983,b65f46d64c688fe98bdbcf93a76a71fc
3,ce110562b3e2f7e5cad3775b32d9caa5,b65f46d64c688fe98bdbcf93a76a71fc
4,8188d20745471273fa69ba44a5b28473,b65f46d64c688fe98bdbcf93a76a71fc
5,e9c621f322640e5f3b3842d19b38aaa8,b65f46d64c688fe98bdbcf93a76a71fc
6,f0c99cfcdbda49dc424848666c6b31a9,b65f46d64c688fe98bdbcf93a76a71fc
7,7de7b4609c6b51d956d27050c958d885,b65f46d64c688fe98bdbcf93a76a71fc
8,05ce4ab49d90139ec1f7af9f0255c9c6,b65f46d64c688fe98bdbcf93a76a71fc
9,051caa763b05e64d4c8699078e942dfd,9c5c8bb82d2cd46fbd7582f944fe370e


## Challenge 2: Create the sparse user-likes matrix 
Create a **sparse matrix** that stores the data for the user-likes associations as described in the paper. The name of the user-likes matrix must be `M`, to keep the naming convention used in the paper. It is important that to store the data in a sparse matrix; you might need to research the topic of using sparse matrices in python (i.e. look at the `scipy.sparse` package and the `csr_matrix` class). *Hint: For this challenge,  you might find  the `merge` method from `pandas` library useful*. 

Once you create the matrix print its shape on the answer summary cell.

In [202]:
r = pd.merge(left=ul, right=users.reset_index(), on='userid', how="left")['index']
c = pd.merge(left=ul, right=likes.reset_index(), on="likeid", how="left")['index']
M = csr_matrix((np.ones((len(ul))), (r,c)))

In [167]:
M.shape


(110728, 1580284)

## Challege 3: Trim the sparse user-likes matrix M.
The paper calls for the matrix `M` to be trimmed (i.e. to remove users which did not like a minimum number of posts and posts that did not receive a minimum number of likes). In this challenge, you are expected to implement the trimming preprocessing step as described in the paper. The resulting matrix must be a sparse matrix. *Hint: you might want to research to index slice sparse matrices in python*

In [208]:
y=users.political
#Trimming the sparse matrix to get rid of minimum number of likes/users who didn't like a minimum number os posts
while True:
    i = M.shape[0] + M.shape[1]
    y = y[np.array(np.sum(M, axis=1) >= 50).flatten()]
    M = M[np.array(np.sum(M, axis=1) >= 50).flatten(), :]
    M = M[:,np.array(np.sum(M, axis=0) >= 150).flatten()]
    if i == (M.shape[0] + M.shape[1]):
        break

In [209]:
y = pd.DataFrame(y).reset_index(drop=True)

In [170]:
M.shape


(19742, 8523)

In [171]:
y.shape

(19742, 1)

## Challenge 4:  Split the sparse matrix into a training and testing matrices
To build a prediction model the paper calls for separating the data matrix (i.e. the user-like matrix) into a training and testing matrices. As part of this challenge, you need to create a training matrix `M_train` that includes the preferences of 80% of the users, and a testing matrix `M_test` that include the preferences of the remaining 20% of the users in the dataset. Both matrices `M_train` and `M_test` must be stored as sparse matrices. User will be randomly assigned to the training or testing set.

Print the shapes of the `M_train` and `M_test` matrices.




In [172]:
#Splitting the matrix into M_train,M_test,y_train,y_test. Using a 80/20 split, random state=15
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
M_train,M_test,y_train,y_test=train_test_split(M,y,test_size=0.2,random_state=15)


In [173]:
M_train.shape

(15793, 8523)

In [174]:
M_test.shape

(3949, 8523)

In [175]:
y_train.shape

(15793, 1)

In [176]:
y_test.shape

(3949, 1)

## Challege 5 : Reduce the dimensionality of the M_train matrix using SVD

In this challenge, you must use singular value decomposition (SVD) to reduce the dimensionality of the training matrix `M_train`. Remember that `M_train` is encoded as a sparse matrix so you would need to use an appropriate implementation of the SVD for sparse matrices. Your output should be the matrices `U`, `S` and `Vt` that correspond to the left singular vectors, the singular values, and the right singular vectors, respectively. You must specify a parameter `k` that defines the number of reduced dimensions you want to keep.

Print the shapes of the `U`, `S` and `Vt` matrices and the value you selected for parameter `k`.



In [177]:
k = 15
U, S, Vt = svds(M_train, k=k, random_state=15)
#Seeing the shape of the different matrices
print("Shape of U:", U.shape)
print("Shape of S:", S.shape)
print("Shape of Vt:", Vt.shape)
print("Value of k:", k)

Shape of U: (15793, 15)
Shape of S: (15,)
Shape of Vt: (15, 8523)
Value of k: 15


## Challege 6 : Use the varimax algorithm to rotate the SVD dimensions. 
Use the varimax algorithm to rotate the SVD dimensions as described in the paper. A good python library that implements varimax algorithm is the `factor_analyzer` and its `Rotator` class (already installed in this notebook for your in the started cell above); i.e. using `rotator = Rotator(method='varimax')`. Store the varimax SVD into the matrices `U_rot` and `V_rot` respectively. *Hint: you can apply varimax of `Vt` to get `V_rot` and then multiply `M` by  `Vt_rot` to obtain the `U_rot` (as done in the paper); you might want to research hot to user the `Rotator` class*. 

Print the shapes of the matrices `U_rot` and `V_rot`.


In [178]:
from factor_analyzer import Rotator
V = Vt.T
rotator = Rotator(method='varimax')
V_rot = rotator.fit_transform(V)
#Vt_rot = V_rot.T
U_rot = M_train.dot(V_rot)

# Print the shapes of U_rot and V_rot
print('Shape of U_rot:', U_rot.shape)
print('Shape of V_rot:', Vt.shape)

Shape of U_rot: (15793, 15)
Shape of V_rot: (15, 8523)


## Challenge 7 : Train two predictive model to predict the political  affilication of users.
Build a predictive model that predicts the political affiliation of the users given the user's post preferences (as those are captured in the paper's dataset). To train your model you can use data from matrix `M_train` only. You are expected to use two different predictive models of your choice (i.e. Knn, SVM, Logistic Regression or some other model of your choice).

As part of this challenge you are expected to:
- Apply any necessary preprocessing to handle missing values.
- The models must be applied on the reduced-dimensional feature generated by your analysis above (i.e. use the singular rotated singular dimensions)
- Use cross-validation for model selection (i.e. to identify optimal parameters for your model, or any meta-parameters)
- Report training and cross-validation performance of the two models as well as the optimal parameters for your model identified by cross validation. 


**FIRST MODEL TRAINING: Use the cells below to train the predictive model using the first model you selected.**

In [179]:
from sklearn.impute import SimpleImputer
#initilizing 
X_train = M_train.dot(Vt.T)
X_test = M_test.dot(Vt.T)

#getting rid of the null values
X_train=pd.DataFrame(X_train)
X_train['y']=y_train
X_train=X_train[X_train['y'].isnull()==False]
X_train['y'].isnull()
y_train=X_train['y']
X_train=X_train.drop('y',axis=1)


X_test=pd.DataFrame(X_test)
X_test['y']=y_test
X_test=X_test[X_test['y'].isnull()==False]
X_test['y'].isnull()
y_test=X_test['y']
X_test=X_test.drop('y',axis=1)

# Check the shapes of the training and test sets
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (3205, 15)
Shape of y_train: (3205,)
Shape of X_test: (231, 15)
Shape of y_test: (231,)


**FIRST MODEL ACCURACY REPORT: Use the cell below to report on the accuracy of your first model**


In [180]:
#Performing SVM
from sklearn import svm
from sklearn.metrics import accuracy_score
model  = svm.SVC(C=1.0,kernel='rbf',gamma='scale')
model.fit(X_train,y_train)
y_pred= model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.7445887445887446


**SECOND MODEL TRAINING: Use the cells below to train the predictive model using the second model you selected.**

In [199]:
#Performing KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
  
n_neighbors = 43  

cv = StratifiedKFold(n_splits=n_splits, random_state=15, shuffle=True)
model=KNeighborsClassifier(n_neighbors=k)
cv_acc=cross_val_score(estimator=model,
                           X=X_train,
                           y=y_train,
                           cv=cv,
                           n_jobs=-1)
scores.append(np.mean(cv_acc))


**SECOND MODEL ACCURACY REPORT: Use the cell below to report on the accuracy of your second model**

In [200]:
print("Accuracy was: {:.2f}".format(max(scores)))

Accuracy was: 0.80


## Challenge 8 Apply your model in the independent test set.
User the cells below to apply your two predictive models on the test dataset `M_test` and report testing accuracy for each model.

As part of this challenge you are expected to:
- Project each observation in the test dataset `M_test` onto the reduced-dimensional features space.
- Apply each on of your model to make predict each user's political preference (for users in the testing set only)
- Report testing accuracy on each of your models.
- Make an assessment as to which model is better and whether either of your models is overfitted or under-fitted.

In [203]:
X=KNeighborsClassifier(n_neighbors=43)
X.fit(X_train,y_train)
y_pred=X.predict(X_test)

In [204]:
X=accuracy_score(y_test,y_pred)
print(X)

0.7445887445887446


In [201]:
model.fit(X_train, y_train)
SVMA = model.score(X_test, y_test)

print("Test accuracy:", SVMA)

Test accuracy: 0.7142857142857143


# END OF EXAM/CODE


*Copyright Statement*: Copyright © 2020 Christoforou. The materials provided by the instructor of this course, including this notebook, are for the use of the students enrolled in the course. Materials are presented in an educational context for personal use and study and should not be shared, distributed, disseminated or sold in print — or digitally — outside the course without permission. You may not, nor may you knowingly allow others to reproduce or distribute lecture notes, course materials  as well as any of their derivatives without the instructor's express written consent.